In [11]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import os

In [1]:
def get_path_list(root_path):
    for idx, name_train in enumerate(os.listdir(root_path)):
        name_train_path = root_path+name_train
        return name_train

In [2]:
def get_class_id(root_path, train_names):
    img_train = []
    img_class = []

    #looping every data in our path and return them as labeled data
    for idx, name_train in enumerate(os.listdir(root_path)):
        name_train_path = root_path+name_train
        for person_name in os.listdir(name_train_path):
            img = name_train_path + '/' + person_name
            img_train.append(img)
            img_class.append(idx)
    return img_train, img_class

In [3]:
def detect_train_faces_and_filter(image_list, image_classes_list):
    train_face_grays = []
    filtered_classes_list = image_classes_list

    # init cascade function to trained images both positive and negative
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    for img in image_list:
        img_gray = cv2.imread(img,0)
        # doing the detection
        detected_face = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)
        if(len(detected_face)<1):
            continue
        for face_rect in detected_face:
            x,y,h,w = face_rect
            # cropping the image to fit the face
            face_img = img_gray[y:y+h, x:x+w] 
            train_face_grays.append(face_img)
    return train_face_grays, filtered_classes_list

In [4]:
def train(train_face_grays, image_classes_list):
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(train_face_grays, np.array(image_classes_list))
    return face_recognizer

In [5]:
def get_test_images_data(test_root_path):
    img_list = []
    for name_test in os.listdir('dataset/test'):
        img_bgr= 'dataset/test/'+name_test
        img_list.append(img_bgr)
    return img_list

In [6]:
def detect_test_faces_and_filter(image_list):
    test_faces_gray = []
    test_faces_rect = []
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    for img in image_list:
        img_bgr = cv2.imread(img)
        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        # perform the cascade detection
        detected_face = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)
        
        if(len(detected_face)<1):
            continue
        for face_rect in detected_face:
            x,y,h,w = face_rect
            face_img = img_gray[y:y+h, x:x+w]
            test_faces_gray.append(face_img)
            test_faces_rect.append(face_rect)

    return test_faces_gray, test_faces_rect

In [7]:
def predict(recognizer, test_image_list):
    predict_result = []
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    for img in test_image_list:
        # detecting every faces in test_image_list
        detected_face = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
        
        if(len(detected_face)<1):
            continue
        for face_rect in detected_face:
            # perform the recognizer to predict the image
            predict = recognizer.predict(img)
            predict_result.append(predict)
    return predict_result

In [8]:
def draw_prediction_results(predict_results, test_image_list, test_faces_rects, train_names, size):
    final_rects = []
    person_name = os.listdir('dataset/train')
    # looping every prediction
    for face_rect,image,pred_result in zip(test_faces_rects,test_image_list,predict_results):
        x,y,h,w = face_rect
        res,conf = pred_result
        img_bgr= cv2.imread(image)
        # calculate the percentage of face detection
        confidence = math.floor(conf *100)/100
        text = person_name[res]+ ' ' + str(confidence) + '%'
        cv2.putText(img_bgr, text, (x,y-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0,255,0) , 2)
        # draw the rectangle with green line
        cv2.rectangle(img_bgr, (x,y), (x+w, y+h), (0,255,0))
        final_rects.append(img_bgr)
    return final_rects

In [9]:
def combine_and_show_result(image_list, size):
    image = [cv2.resize(img,(size,size)) for img in image_list]
    concated_img = cv2.hconcat(image)
    cv2.imshow('result', concated_img)
    cv2.waitKey(0)

In [ ]:
if __name__ == "__main__":
    
    train_root_path = 'dataset/train/'

    train_names = get_path_list(train_root_path)
    train_image_list, image_classes_list = get_class_id(train_root_path, train_names)
    train_face_grays, filtered_classes_list = detect_train_faces_and_filter(train_image_list, image_classes_list)
    recognizer = train(train_face_grays, filtered_classes_list)

    test_root_path =  'dataset/test/'

    test_image_list = get_test_images_data(test_root_path)
    test_faces_gray, test_faces_rects = detect_test_faces_and_filter(test_image_list)
    predict_results = predict(recognizer, test_faces_gray)
    predicted_test_image_list = draw_prediction_results(predict_results, test_image_list, test_faces_rects, train_names, 200)
    
    combine_and_show_result(predicted_test_image_list, 200)